# LangChain   

## 1. Chains
Chains in LangChain are sequences of components that process language input and generate responses. These components can be simple functions, calls to language models, or more complex operations such as database queries or API calls. The idea is to chain these components together to form a pipeline that handles the entire process of understanding, processing, and responding to user inputs.

__Types of Chains__:  
__Linear Chains__: Execute a sequence of tasks one after another, where the output of one component is the input to the next.  
__Branching Chains__: Allow for conditional execution paths depending on the input or the outcomes of previous tasks in the chain.  
__Looping Chains__: Enable repeating a set of tasks until a certain condition is met, useful for refining answers or gathering additional information.  

https://python.langchain.com/docs/modules/chains/

## 2. Agents
Agents in LangChain are entities that manage the interaction between the user and the system. An agent coordinates the chains, decides which one to invoke based on the context or the nature of the query, and manages state across interactions. This is crucial for maintaining the continuity in conversations, especially in applications like chatbots or virtual assistants.

__Key Features of Agents__:
__Context Management__: Agents maintain a conversational context, remembering past interactions and using this context to make decisions about future interactions.  
__Stateful Logic__: They can manage stateful interactions, which are essential for tasks that require an understanding of the conversation history to provide relevant responses.  
__Modularity__: Agents are designed to be modular, so developers can swap out components or chains as needed without redesigning the entire system.  

https://python.langchain.com/docs/modules/agents/
## 3. Retrieval Strategies
Retrieval strategies in LangChain define how the system retrieves information needed to answer queries or perform tasks. This is particularly important when the system needs to pull in external information from databases, APIs, or through search queries.

__Examples of Retrieval Strategies__:  
__Document Retrieval__: Fetching relevant documents or content from a database or a set of files based on the query.  
__API Retrieval__: Making API calls to external services to retrieve data or perform actions relevant to the user’s request.  
__Search-Augmented Models__: Integrating search results into the processing chain to enhance the model’s responses with up-to-date or specific information not contained within the model’s training data.  
### Integration and Operation
In LangChain, these components are integrated into a cohesive system where agents use chains to handle user inputs, applying appropriate retrieval strategies as needed. The system is designed to be flexible and scalable, allowing developers to customize each component according to their specific requirements.

### For example, a LangChain application for customer support might use:

A chain that includes components for greeting users, understanding their problems, and retrieving solutions from a knowledge base.  
An agent that manages these chains, deciding when to escalate an issue to a human based on the complexity of the problem.  
A retrieval strategy that pulls information from both a FAQ database and real-time product data to provide accurate and relevant responses.  
This architecture allows LangChain to be both powerful and adaptable, making it an excellent choice for developers looking to build advanced language-driven applications.  


# SQL QUERY CHAIN

In [7]:
# pip install -U langchain langchain-community langchain-openai
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase
import streamlit as st

link to the dataset https://www.sqlitetutorial.net/sqlite-sample-database/

In [8]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.get_usable_table_names())

['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


In [9]:
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=st.secrets["OPENAI_API_KEY"])
chain = create_sql_query_chain(llm, db)
generated_sql_query = chain.invoke({"question": "How many employees are there"})
print(generated_sql_query)
db.run(generated_sql_query)

SELECT COUNT("EmployeeId") AS total_employees
FROM employees;


'[(8,)]'

In [10]:
def execute_query(question):
    """
    Executes a generated SQL query based on the provided question.

    Parameters:
    question (str): The natural language question from which to generate an SQL query.

    Returns:
    list: The result of the SQL query.
    """
    # Ensure that the chain and the necessary components are correctly initialized
    chain = create_sql_query_chain(llm, db)
    
    # Invoke the chain to generate an SQL query
    generated_sql_query = chain.invoke({"question": question})
    
    # Execute the generated SQL query and return the results
    return db.run(generated_sql_query)

In [12]:
execute_query("Who are all my employees?")

"[(1, 'Andrew', 'Adams'), (2, 'Nancy', 'Edwards'), (3, 'Jane', 'Peacock'), (4, 'Margaret', 'Park'), (5, 'Steve', 'Johnson'), (6, 'Michael', 'Mitchell'), (7, 'Robert', 'King'), (8, 'Laura', 'Callahan')]"

In [16]:
execute_query("Where are the top 5 most popular genres?")

"[('Rock', 1297), ('Latin', 579), ('Metal', 374), ('Alternative & Punk', 332), ('Jazz', 130)]"

In [19]:
execute_query("How much does the average customer spend? Round to two decimal places.")

'[(5.65,)]'